# Описание задачи.

Мы – организаторы некой многопользовательской онлайн-игры. На каждого игрока мы ведем игровую статистику, которая представлена <br/>
в виде 127 игровых показателей. Каждый показатель характеризует поведение игрока в одной разновидности игровых ситуаций. <br/>
Показатель является отношением числа произведенных игровых действий одного типа к числу возможностей данное действие произвести. <br/>

У нас уже есть группа игроков (первая часть таблицы, все игроки с именами  gr_player…) с одинаковой игрой. <br/>
Какими диапазонами показателей характеризуется стиль игры данной группы? <br/>
В ответе требуется в любой форме указать алгоритм отнесения произвольного игрока по его игровым показателям к данной группе, а также <br/>
продемонстрировать данный алгоритм при ответе на вопрос: кого из на указанных игроков из второй части таблицы (зеленая часть, имена new_player…) можно отнести к данной группе и <br/>
почему нельзя остальных? 
    
    

# Содержание

1. EDA
    * Первичный анализ данных 
    * Преобразование данных
    * Standart-Scaling 
<br/>
2. Описание алгоритма
<br/>
3. Neighbors
    * Преобразование данных
    * Обучение модели
    * Предсказание модели
    * Собираем финальную таблицу
<br/>
4. Итоговые результаты
5. Что можно улучшить


In [86]:
import pandas as pd
import numpy as np



from sklearn import preprocessing
from sklearn import decomposition

from sklearn import metrics
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE


from sklearn import datasets
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.cluster import AffinityPropagation
from sklearn.cluster import DBSCAN
from sklearn.cluster import SpectralClustering
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import MeanShift

from sklearn.neighbors import NearestNeighbors


from functools import reduce

import seaborn as sns
sns.set(style="white", color_codes=True)
import matplotlib as mpl
import matplotlib.pyplot as plt


import plotly
import plotly.graph_objs as go
import plotly.express as px


import warnings
warnings.filterwarnings("ignore")

# show plots inline
%matplotlib inline

seed = 42
np.random.seed(42)

In [87]:
df = pd.read_csv('DS_task_2_Group_Style_By_stats_v2.csv')

df.head()

,player,stat1,stat2,stat3,stat4,stat5,stat6,stat7,stat8,stat9,...,stat118,stat119,stat120,stat121,stat122,stat123,stat124,stat125,stat126,stat127
0,gr_player1,0.000000,0.551756,NaN,0.227376,0.089159,NaN,0.625330,NaN,0.080153,...,0.450982,0.205062,0.326149,0.255466,0.294282,0.002488,NaN,0.265898,NaN,NaN
1,gr_player2,0.000000,0.559532,NaN,0.197149,0.109987,NaN,0.685624,NaN,0.120219,...,0.438461,0.223621,0.325301,0.274218,0.289307,0.013705,NaN,0.251024,NaN,NaN
2,gr_player3,0.000013,0.542488,NaN,0.193920,0.079732,NaN,0.619612,NaN,0.072533,...,0.443868,0.197362,0.323240,0.256494,0.303659,0.005041,NaN,0.258904,NaN,NaN
3,gr_player4,0.000000,0.560295,NaN,0.228996,0.078283,NaN,0.626145,NaN,0.049505,...,0.439452,0.201277,0.311881,0.253795,0.292665,0.005589,NaN,0.244630,NaN,NaN
4,gr_player5,0.000102,0.592927,NaN,0.231728,0.057743,NaN,0.597334,NaN,0.058140,...,0.426566,0.202446,0.322222,0.267663,0.296040,0.004804,NaN,0.255882,NaN,NaN


# EDA

### Первичный анализ данных

In [88]:
# Посмотрим размер датасета и пропущенные значения.
print("Размер датасета:", df.shape)

Размер датасета: (53, 128)


In [89]:
# Проверим, есть ли в нашем датасете пропуски
sum(df.isnull().sum())

1551

In [90]:
# Посмотрим статистические значения переменных

df.describe()

,stat1,stat2,stat3,stat4,stat5,stat6,stat7,stat8,stat9,stat10,...,stat118,stat119,stat120,stat121,stat122,stat123,stat124,stat125,stat126,stat127
count,53.000000,53.000000,0.0,53.000000,53.000000,0.0,53.000000,0.0,53.000000,0.0,...,53.000000,53.000000,53.000000,53.000000,53.000000,53.000000,0.0,53.000000,0.0,0.0
mean,0.000027,0.537849,NaN,0.210166,0.083576,NaN,0.616679,NaN,0.087899,NaN,...,0.450390,0.207702,0.322590,0.267999,0.297837,0.009250,NaN,0.264232,NaN,NaN
std,0.000043,0.043630,NaN,0.025430,0.018249,NaN,0.028060,NaN,0.028746,NaN,...,0.020245,0.011341,0.049068,0.012169,0.029435,0.007855,NaN,0.010712,NaN,NaN
min,0.000000,0.425349,NaN,0.165808,0.045775,NaN,0.547669,NaN,0.030928,NaN,...,0.392911,0.178428,0.206704,0.242901,0.165283,0.000000,NaN,0.237372,NaN,NaN
25%,0.000000,0.513078,NaN,0.193920,0.069825,NaN,0.600530,NaN,0.066667,NaN,...,0.438461,0.201277,0.297872,0.259350,0.288689,0.003559,NaN,0.258438,NaN,NaN
50%,0.000000,0.542488,NaN,0.210689,0.081081,NaN,0.620268,NaN,0.088235,NaN,...,0.445217,0.207946,0.318480,0.267369,0.298731,0.007303,NaN,0.263282,NaN,NaN
75%,0.000037,0.561430,NaN,0.228403,0.097610,NaN,0.633764,NaN,0.106383,NaN,...,0.462004,0.215279,0.343915,0.275611,0.314806,0.012301,NaN,0.270290,NaN,NaN
max,0.000188,0.623913,NaN,0.284902,0.118987,NaN,0.685624,NaN,0.148515,NaN,...,0.503048,0.232718,0.501706,0.295311,0.346110,0.034413,NaN,0.287544,NaN,NaN


**Вывод:** По этой таблице можно сказать, что <br/> 
1) Часть переменных имеет значения NaN во всем столбце, тк count = 0, <br/> а значения min, max, mean равны NaN. 
Если в столбце содержится одиноаковое значение для всех элементов, то такие столбцы можно <br/> 
выкидывать из рассмотрения, тк они не несут никакой информации.

2) Есть столбцы с частью пропущенных значений, например столбец "stat9" в нем есть два пропуска. <br/> 
Это опять же можно увидеть по строке "count" <br/> 

### Преобразование данных

In [91]:
# Получим список колонок, в которых стоит только NaN, далее их просто удалим
drop_columns = pd.DataFrame(df.describe().iloc[0] == 0).reset_index().rename(columns={"index": "column", "count": "value"})
drop_columns = drop_columns[drop_columns['value']==True][['column']].values
drop_columns = [column[0] for column in drop_columns.tolist()]
print("Count of drop columns: ", len(drop_columns))

# Удалим эти колонки
df.drop(drop_columns, axis=1, inplace=True)

Count of drop columns:  26


In [92]:
# Тк в данных есть еще NaN, их нужно заменить на какое-то значение, 
# для этого проверим какое минимальное значение в текущем DataFrame

df.describe().min().min()

0.0

**Вывод** Значит оставшиеся пропущенные значения можно заменить на -1, те выбрать какую-то величину, которая не появлялась в датасете

In [93]:
# Заполним пропущенные значения -1

df.fillna(-1, inplace = True)

In [94]:
# Произведем поиск столбцов, которые имеют одно и тоже значение для всех игроков.
# Такие столбцы никак не влияют на результат. Возможно это просто поломка показателя, надо обращаться 

drop_columns_list = []

for column in df.drop(['player'], axis = 1):
    if max(df[column]) == min(df[column]) == np.mean(df[column]):
        drop_columns_list.append(column)
        
drop_columns_list

[]

**Вывод:** В датафрейме нет столбцов, которые состоят из одинакового значения!

### Standart-Scaling

Нужно отшкалировать данные, чтобы элементы с большим значением не вносили более сильный вклад в подсчет расстояния.

In [95]:
X = df.copy(deep=True)

In [96]:
#list for cols to scale

drop_columns = ['player']

cols_to_scale = X.drop(drop_columns, axis=1).columns

#create and fit scaler
scaler = preprocessing.StandardScaler()
scaler.fit(X[cols_to_scale])

#scale selected data
X[cols_to_scale] = scaler.transform(X[cols_to_scale])

X_scaled = X.copy()
X_scaled.head(5)

,player,stat1,stat2,stat4,stat5,stat7,stat9,stat11,stat12,stat13,...,stat113,stat115,stat117,stat118,stat119,stat120,stat121,stat122,stat123,stat125
0,gr_player1,-0.643294,0.321794,0.683234,0.308838,0.311244,-0.272071,0.701357,0.524680,0.127595,...,0.365707,1.159044,0.340671,0.029536,-0.235059,0.073241,-1.039729,-0.121927,-0.869000,0.157011
1,gr_player2,-0.643294,0.501723,-0.516799,1.461084,2.480596,1.135058,1.770684,-1.344437,-0.922648,...,0.276816,-0.142419,-1.242781,-0.594886,1.417085,0.055789,0.515992,-0.292579,0.572543,-1.244763
2,gr_player3,-0.339109,0.107353,-0.644986,-0.212669,0.105496,-0.539688,-0.023666,-0.238948,0.431117,...,0.074405,-0.254164,0.638718,-0.325232,-0.920503,0.013379,-0.954422,0.199680,-0.540925,-0.502175
3,gr_player4,-0.643294,0.519396,0.747526,-0.292849,0.340568,-1.348431,-0.348113,1.073360,-0.574958,...,0.552998,0.211529,-0.643096,-0.545467,-0.571967,-0.220326,-1.178363,-0.177397,-0.470449,-1.847415
4,gr_player5,1.747659,1.274476,0.855986,-1.429150,-0.696049,-1.045181,-0.710141,-0.146971,-0.107954,...,1.304310,0.835729,-0.342423,-1.188053,-0.467919,-0.007561,-0.027868,-0.061640,-0.571374,-0.786933


# 2. Описание алгоритма

В качестве алгоритма нахождения похожих групп пользователей выберем алгоритм K-ближайших соседей.<br/>
Суть его в том, что когда приходит новый элемент, то строится расстояние от него до всех имеющихся элементов и <br/>
далее выбираются K ближайших

В качестве расстояния можно использовать много различных метрик, в текущей задаче будем использовать просто Евклидову метрику.

# Neighbors

### Преобразование данных

In [97]:
# Выделим таблицу обучения,  а также новые примеры для предсказания

X = X_scaled.head(49)
X_new = X_scaled.tail(4)

In [98]:
# Создадим словарик, где сохраним связку индексов и названия группы.
# Это пригодится в будущем дляобратного маппинга

map_dict = X['player'].to_dict()

### Обучение модели

In [99]:
neigh = NearestNeighbors(metric='euclidean', n_neighbors=3, radius=1)
neigh.fit(X.drop(['player'], axis=1))


NearestNeighbors(metric='euclidean', n_neighbors=3, radius=1)

### Предсказание модели

In [100]:
distance , index = neigh.kneighbors(X_new.drop(['player'], axis=1), 3, return_distance=True)

print("distance", distance)
print("index", index)

distance [[20.69304066 21.78609278 22.08881159]
 [ 7.26540874  7.60509599  7.79530014]
 [ 9.35548781  9.47698612  9.49790543]
 [20.41614781 21.88533423 22.16997102]]
index [[42 24 39]
 [48  6 37]
 [24 43 16]
 [42 39 36]]


### Собираем финальную таблицу

In [101]:
neighbors = pd.DataFrame(index, columns = ["neighbor_1", "neighbor_2", "neighbor_3"])
distances = pd.DataFrame(distance, columns = ["distance_1", "distance_2", "distance_3"])

result = pd.concat([X_new['player'].reset_index(drop=True), neighbors, distances],  axis=1)


# По индексам восстаналиваем группу пользователя
result['neighbor_1'] = result['neighbor_1'].map(map_dict)
result['neighbor_2'] = result['neighbor_2'].map(map_dict)
result['neighbor_3'] = result['neighbor_3'].map(map_dict)

result

,player,neighbor_1,neighbor_2,neighbor_3,distance_1,distance_2,distance_3
0,new_player1,gr_player43,gr_player25,gr_player40,20.693041,21.786093,22.088812
1,new_player2,gr_player49,gr_player7,gr_player38,7.265409,7.605096,7.795300
2,new_player3,gr_player25,gr_player44,gr_player17,9.355488,9.476986,9.497905
3,new_player4,gr_player43,gr_player40,gr_player37,20.416148,21.885334,22.169971


**Вывод:** В финальном ответе мы получили, ближайших соседей и расстояние до них. 

In [77]:
neighbors = pd.DataFrame(index, columns = ["neighbor_1", "neighbor_2", "neighbor_3"])
distances = pd.DataFrame(distance, columns = ["distance_1", "distance_2", "distance_3"])

result = pd.concat([X_new['player'].reset_index(drop=True), neighbors, distances],  axis=1)


# По индексам восстаналиваем группу пользователя
result['neighbor_1'] = result['neighbor_1'].map(map_dict)
result['neighbor_2'] = result['neighbor_2'].map(map_dict)
result['neighbor_3'] = result['neighbor_3'].map(map_dict)

result

,player,neighbor_1,neighbor_2,neighbor_3,distance_1,distance_2,distance_3
0,new_player1,gr_player28,gr_player42,gr_player48,0.732752,0.758433,0.760368
1,new_player2,gr_player38,gr_player49,gr_player29,0.357238,0.360554,0.365438
2,new_player3,gr_player3,gr_player9,gr_player16,0.335044,0.424540,0.470379
3,new_player4,gr_player8,gr_player39,gr_player11,0.698912,0.736396,0.776414


# 4. Итоговые результаты

In [102]:
result.to_csv('Task_2_answer.csv',index=False)

# 5. Что можно улучшить

1. **Ансамбль алгоритмов** <br/>
Можно было использовать ансамбль алгоритмов, те посчитать K ближайших соседей с помощью нескольких разных метрик и
далее выбрать среди первого соседа наиболее встречающегося, среди в второго и тд... <br/>
<br/>

2. **Embeddings** <br/>
Если бы нам были даные изначальные данные по пользователям и группы к которым они относятся, то возможно было бы построить более лучшие embedding групп с помощью архитектуры нейронной сети: Coder-Decoder. После этих embeddings алгоритм K-ближайших соседей работал бы намного лучше.
